In [1]:
import csv, os
import matplotlib.pyplot as plt
import numpy as np
def read_csv(dir, run, d=1):
    filename = os.path.join(dir, run, "logs.csv")
    steps, eps_ret, eps_n = [], [], []        
    with open(filename, 'r') as file:
        csvreader = csv.reader((line.replace('\0','') for line in file))        
        header = next(csvreader)
        for row in csvreader:            
            if len(row) > 0 and int(row[-1]) > 100:
                steps.append(int(row[2])/d)
                eps_ret.append(float(row[-2]))
                #eps_n.append(row[-1])
    return steps, eps_ret

def simplify(steps, eps_ret, gap=100000):
    ind = 0
    new_steps, new_eps_ret = [], []
    for n, (i, j) in enumerate(zip(steps, eps_ret)):
        if i > ind:
            new_steps.append(i)
            new_eps_ret.append(j)
            ind += gap
    return new_steps, new_eps_ret
    
def mv(a, n=1000) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n    

In [95]:
servers = ["uk",
           "cam",
           "v1", 
           "v2", 
           "v3",
           "v4",
           "v5"
           ]
latest_runs = ["alstm_3_1_rec_n_10_rei_aug2_clip_mem1_imcost_0_perfect",
               "alstm_3_1_rec_n_10_rei_aug2_clip_mem1_imcost_0.5_perfect",
               "alstm_3_1_rec_n_10_rei_aug2_clip_mem1_imcost_1_perfect",
               "alstm_3_1_rec_n_10_rei_aug2_clip_mem1_imcost_0",
               "alstm_3_1_rec_n_10_rei_aug2_clip_mem1_imcost_0.5",
               "alstm_3_1_rec_n_10_rei_aug2_clip_mem1_imcost_1",
               "alstm_3_1_rec_n_10_rei_aug2_clip_mem0",
               
              ]               

for server, lr in zip(servers, latest_runs):
    !mkdir ~/RS/thinker/logs/planner_logs/$lr
    !scp $server:~/RS/thinker/logs/torchbeast/latest/*.* ~/RS/thinker/logs/planner_logs/$lr
        
runs = os.listdir('logs/planner_logs')
steps, eps_rets = {}, {}
mv_avg = 20

for run in runs:
    if "past" == run: continue
    if "rei" in run:
        st = run.find("rec_n_")
        d = int(run[st+6:run.find("_", st+6)])
    else:
        d = 1  
    step, eps_ret = simplify(*read_csv('logs/planner_logs', run, d))
    steps[run] = step
    eps_rets[run] = eps_ret                        

mkdir: cannot create directory ‘/home/sc/RS/thinker/logs/planner_logs/alstm_3_1_rec_n_10_rei_aug2_clip_mem1_imcost_0_perfect’: File exists
fields.csv                                    100%    0     0.0KB/s   00:00    
logs.csv                                      100% 3401KB 100.3MB/s   00:00    
meta.json                                     100% 6083    12.7MB/s   00:00    
model.tar                                     100% 4715KB 101.1MB/s   00:00    
out.log                                       100%  453     1.5MB/s   00:00    
mkdir: cannot create directory ‘/home/sc/RS/thinker/logs/planner_logs/alstm_3_1_rec_n_10_rei_aug2_clip_mem1_imcost_0.5_perfect’: File exists
fields.csv                                    100%    0     0.0KB/s   00:00    
logs.csv                                      100% 5344KB  12.0MB/s   00:00    
meta.json                                     100% 5039   206.6KB/s   00:00    
model.tar                                     100% 4723KB  15.0MB/s   00:00    


In [70]:
runs = os.listdir('logs/planner_logs')
steps, eps_rets = {}, {}
mv_avg = 20

for run in runs:
    if "past" == run: continue
    if "rei" in run:
        st = run.find("rec_n_")
        d = int(run[st+6:run.find("_", st+6)])
    else:
        d = 1  
    step, eps_ret = simplify(*read_csv('logs/planner_logs', run, d))
    steps[run] = step
    eps_rets[run] = eps_ret                  

In [4]:
#%matplotlib inline
%matplotlib qt

def plot(steps, eps_rets, plot_dict=None, mv_avg=20, xlim=int(1e8)):
    if plot_dict is None:
        plot_dict = {k:k for k in steps.keys()}
    linestyles=["solid", "dashed", "dotted","dashdot"]
    f, ax = plt.subplots(1, 1, figsize=(12,7))
    n = 0
    for run, label in plot_dict.items():
        step = steps[run]
        eps_ret = eps_rets[run]
        bound = np.nonzero(np.array(step) > xlim)[0]
        if len(bound) >= 1:
            step = step[:bound[0]]
            eps_ret = eps_ret[:bound[0]]
        ax.plot(np.array(step[mv_avg-1:]), 
                mv(eps_ret, n=mv_avg), 
                label=label, 
                linestyle=linestyles[n//10],
                linewidth=1)
        n += 1
    plt.xlabel('Step')
    plt.ylabel('Eps return')
    plt.axhline(y=0.12, color='r', label="base policy", linewidth=1)
    plt.axhline(y=1.08, color='g', label="1-step exhaustive search", linewidth=1)
    plt.axhline(y=1.17, color='y', label="2-step exhaustive search", linewidth=1)
    plt.axhline(y=1.59, color='b', label="2-step 15-rollout search", linewidth=1)
    plt.axhline(y=1.50, color='c', label="20-simulations MCTS", linewidth=1)
    plt.tight_layout() 
    plt.legend()
    plt.show()   
    
plot(steps, eps_rets)    

In [48]:
plot_dict = {#"lstm_3_1_rec_n_5_gbste": "LSTM_3_1",
             #"tran_3_1_rec_n_5_gbste_aug": "Transformer",        
             #"alstm_3_1_rec_n_5_gbste_aug": "ALSTM_3_1 (4 steps)",
             #"alstm_3_1_rec_n_10_gbste_aug": "ALSTM_3_1 (9 steps)",
             "alstm_3_1_rec_n_1_gbste_aug_clip": "ALSTM_3_1 (0 steps; clipped)",
             #"alstm_3_1_rec_n_2_gbste_aug_clip": "ALSTM_3_1 (1 steps; clipped)",
             "alstm_3_1_rec_n_5_gbste_aug_clip": "ALSTM_3_1 (4 steps; clipped)",             
             "alstm_3_1_rec_n_10_gbste_aug_clip": "ALSTM_3_1 (9 steps; clipped)",
             #"alstm_3_2_rec_n_5_gbste_aug_clip": "ALSTM_3_2 (4 steps; clipped)",
             #"alstm_3_1_rec_n_5_gbste_aug_clip_large": "ALSTM_3_1 (4 steps; clipped; large)",
             #"alstm_3_1_rec_n_1_gbste_aug_clip_nomem": "ALSTM_3_1 (0 steps; clipped; no_mem)",
             #"alstm_3_1_rec_n_5_gbste_aug_clip_nomem": "ALSTM_3_1 (4 steps; clipped; no_mem)",
             #"alstm_3_1_rec_n_10_gbste_aug_clip_nomem": "ALSTM_3_1 (9 steps; clipped; no_mem)",
             #"alstm_3_1_rec_n_20_gbste_aug_clip_nomem": "ALSTM_3_1 (19 steps; clipped; no_mem)",
             #"alstm_3_1_rec_n_10_gbste_aug_clip_nomem_altmodel": "ALSTM_3_1 (9 steps; clipped; no_mem; altmodel)",
             "alstm_3_1_rec_n_1_gbste_aug_clip_realnomem": "ALSTM_3_1 (0 steps; clipped; real_no_mem)",
             "alstm_3_1_rec_n_5_gbste_aug_clip_realnomem": "ALSTM_3_1 (4 steps; clipped; real_no_mem)",
             "alstm_3_1_rec_n_10_gbste_aug_clip_realnomem": "ALSTM_3_1 (9 steps; clipped; real_no_mem)",
            }
plot(steps, eps_rets, plot_dict, mv_avg=20, xlim=int(10e7))

In [93]:
plot_dict = {"alstm_3_1_rec_n_5_rei_aug_clip_realnomem": "ALSTM_3_1 REI (4 steps)",
             "alstm_3_1_rec_n_10_rei_aug_clip_realnomem": "ALSTM_3_1 REI (9 steps)",
             "alstm_3_1_rec_n_10_rei_aug_clip_realnomem_large": "ALSTM_3_1 REI (9 steps; large)",
             "alstm_3_1_rec_n_5_gbste_aug_clip_realnomem": "ALSTM_3_1 (4 steps)",
             "alstm_3_1_rec_n_10_gbste_aug_clip_realnomem": "ALSTM_3_1 (9 steps)",             
             "alstm_3_1_rec_n_5_rei_clip_realnomem": "ALSTM_3_1 (4 steps; no aug)",
             "alstm_3_1_rec_n_10_rei_clip_realnomem": "ALSTM_3_1 (9 steps; no aug)",
             "alstm_3_1_rec_n_10_rei_clip_realnomem_2": "ALSTM_3_1 (9 steps; no aug) 2",
             "alstm_3_1_rec_n_10_rei_aug1.5_clip_realnomem_large": "ALSTM_3_1 (9 steps; 1.5 aug)",
             "alstm_3_1_rec_n_10_rei_aug2_clip_realnomem_large": "ALSTM_3_1 (9 steps; 2 aug)",
             }
plot(steps, eps_rets, plot_dict, mv_avg=20, xlim=int(10e7))

In [96]:
plot_dict = {
   "alstm_3_1_rec_n_10_rei_aug2_clip_mem1_imcost_0_perfect": "perfect, imcost_0",
   "alstm_3_1_rec_n_10_rei_aug2_clip_mem1_imcost_0.5_perfect": "perfect, imcost_0.5",
   "alstm_3_1_rec_n_10_rei_aug2_clip_mem1_imcost_1_perfect": "perfect, imcost_1",
   "alstm_3_1_rec_n_10_rei_aug2_clip_mem1_imcost_0": "imcost_0",
   "alstm_3_1_rec_n_10_rei_aug2_clip_mem1_imcost_0.5": "imcost_0.5",
   "alstm_3_1_rec_n_10_rei_aug2_clip_mem1_imcost_1": "imcost_1",
    
}

plot_dict["alstm_3_1_rec_n_10_rei_aug_clip_realnomem"] = "base_aug1"
plot_dict["alstm_3_1_rec_n_10_rei_aug2_clip_realnomem_large"] = "base_aug2a"
plot_dict["alstm_3_1_rec_n_10_rei_aug2_clip_mem0"] = "base_aug2b"
plot(steps, eps_rets, plot_dict, mv_avg=20, xlim=int(10e7))

In [ ]:
plot_dict = {"lstm_3_1_rec_n_5_w": "Backprop",
             "lstm_3_1_rec_n_5_gbste": "GB-STE",
             "lstm_3_1_rec_n_5_ste": "STE",
              "lstm_3_1_rec_n_5_rei": "REINFORCE",        
            }
plot(steps, eps_rets, plot_dict)

plot_dict = {"lstm_3_1_rec_n_5_gbste": "GB-STE",
             "lstm_3_1_rec_n_5_gbste_aug": "GB-STE w/ augmented stat.",
             "lstm_3_1_rec_n_5_rei": "REINFORCE",        
             "lstm_3_1_rec_n_5_rei_aug": "REINFORCE w/ augmented stat.",
            }
plot(steps, eps_rets, plot_dict)

plot_dict = {"lstm_3_1_rec_n_5_gbste": "LSTM",
             "tran_3_1_rec_n_5_gbste_aug": "Transformer",        
             "alstm_3_1_rec_n_5_gbste_aug": "LSTM w/ attn. (4 steps)",
             "alstm_3_1_rec_n_10_gbste_aug": "LSTM w/ attn. (9 steps)",          
            }
plot(steps, eps_rets, plot_dict)

plot_dict = {"lstm_3_1_rec_n_5_gbste": "LSTM_3_1",
             "tran_3_1_rec_n_5_gbste_aug": "Transformer",        
             "alstm_3_1_rec_n_5_gbste_aug": "ALSTM_3_1 (4 steps)",
             "alstm_3_1_rec_n_10_gbste_aug": "ALSTM_3_1 (9 steps)",             
             "alstm_3_1_rec_n_5_gbste_aug_clip": "ALSTM_3_1 (4 steps; clipped)",             
             "alstm_3_1_rec_n_10_gbste_aug_clip": "ALSTM_3_1 (9 steps; clipped)",
            }
plot(steps, eps_rets, plot_dict, mv_avg=20, xlim=int(10e7))

plot_dict = {"alstm_3_1_rec_n_1_gbste_aug_clip": "ALSTM_3_1 (0 steps; clipped)",             
             "alstm_3_1_rec_n_2_gbste_aug_clip": "ALSTM_3_1 (1 steps; clipped)",                     
             "alstm_3_1_rec_n_5_gbste_aug_clip": "ALSTM_3_1 (4 steps; clipped)",             
             "alstm_3_1_rec_n_10_gbste_aug_clip": "ALSTM_3_1 (9 steps; clipped)",
            }
plot(steps, eps_rets, plot_dict, mv_avg=20, xlim=int(10e7))

plot_dict = {"alstm_3_1_rec_n_1_gbste_aug_clip": "ALSTM_3_1 (0 steps)",                  
             "alstm_3_1_rec_n_5_gbste_aug_clip": "ALSTM_3_1 (4 steps)",                          
             "alstm_3_1_rec_n_10_gbste_aug_clip": "ALSTM_3_1 (9 steps)",    
             "alstm_3_1_rec_n_1_gbste_aug_clip_realnomem": "ALSTM_3_1 (0 steps; no mem.)",                  
             "alstm_3_1_rec_n_5_gbste_aug_clip_realnomem": "ALSTM_3_1 (4 steps; no mem.)",                          
             "alstm_3_1_rec_n_10_gbste_aug_clip_realnomem": "ALSTM_3_1 (9 steps; no mem.)",
            }
plot(steps, eps_rets, plot_dict, mv_avg=20, xlim=int(5e7))